In [38]:
import numpy as np
from scipy.optimize import minimize
from openptv_python.epi import img_coord
from openptv_python.calibration import Calibration
from openptv_python.parameters import MultimediaPar, ControlPar
from openptv_python.calibration import Exterior, Glass, Interior, ap_52


In [39]:
# Based on the C tests in liboptv/check_orientation.c

from openptv_python.imgcoord import image_coordinates
from openptv_python.parameters import OrientPar, read_control_par
from openptv_python.tracking_frame_buf import TargetArray
from openptv_python.trafo import arr_metric_to_pixel


control_file_name = "testing_folder/corresp/control.par"
# control = ControlPar(4)
control = read_control_par(control_file_name)

orient_par_file_name = "testing_folder/corresp/orient.par"
orient_par = OrientPar().from_file(orient_par_file_name)

cal = Calibration().from_file(
    "testing_folder/calibration/cam1.tif.ori",
    "testing_folder/calibration/cam1.tif.addpar",
)
orig_cal = Calibration().from_file(
    "testing_folder/calibration/cam1.tif.ori",
    "testing_folder/calibration/cam1.tif.addpar",
)


# def test_external_calibration(self):
#     """External calibration using clicked points."""
#     ref_pts = np.array(
#         [
#             [-40.0, -25.0, 8.0],
#             [40.0, -15.0, 0.0],
#             [40.0, 15.0, 0.0],
#             [40.0, 0.0, 8.0],
#         ]
#     )

#     # Fake the image points by back-projection
#     targets = arr_metric_to_pixel(
#         image_coordinates(ref_pts, cal, control.mm),
#         control,
#     )

#     # Jigg the fake detections to give raw_orient some challenge.
#     targets[:, 1] -= 0.1

#     # assertTrue(external_calibration(cal, ref_pts, targets, control))
#     np.testing.assert_array_almost_equal(
#         cal.get_angles(), orig_cal.get_angles(), decimal=3
#     )
#     np.testing.assert_array_almost_equal(
#         cal.get_pos(), orig_cal.get_pos(), decimal=3
#     )

"""Full calibration using clicked points."""
ref_pts = np.array(
    [
        a.flatten()
        for a in np.meshgrid(np.r_[-60:-30:4j], np.r_[0:15:4j], np.r_[0:15:4j])
    ]
).T

# Fake the image points by back-projection
targets = arr_metric_to_pixel(
    image_coordinates(ref_pts, cal, control.mm),
    control,
)

# # Full calibration works with TargetArray objects, not NumPy.
# target_array = TargetArray(len(targets))
# for i, trgt in enumerate(target_array):
#     trgt.set_pnr(i)
#     trgt.set_pos(targets[i])

# Perturb the calibration object, then compore result to original.
cal.set_pos(cal.get_pos() + np.r_[15.0, -15.0, 15.0])
# cal.set_angles(cal.get_angles() + np.r_[-0.5, 0.5, -0.5])

In [40]:
def unflat_cal(flat_cal: np.ndarray):
    # print(Exterior(*flat_cal[:6]))
    # print(Interior(*flat_cal[6:9]))
    # print(ap_52(*flat_cal[9:]))
    
    return Calibration(
        Exterior(*flat_cal[:6]),
        Interior(*flat_cal[6:9]),
        Glass(0,0,1),
        ap_52(*flat_cal[9:]) 
    )
        
def cost_function(flat_cal: np.ndarray, fix: np.ndarray, pix: np.ndarray, cpar: ControlPar):
    # Calculate the difference between predicted and actual 2D points
    # predicted_points = np.array([predict_2d_points(flat_cal, cpar, point) for point in fix])
    cal = unflat_cal(flat_cal)
    predicted_points = arr_metric_to_pixel(
        image_coordinates(fix, cal, cpar.mm),
        cpar,
    )
    # print(pix)
    # print(predicted_points)
    residuals = pix - predicted_points
    # print(residuals)
    
    # Return the sum of squared residuals
    return np.sum(np.sum(residuals**2, axis=1))

# Define the optimization function for Scipy
def optimize_function(flat_cal: np.ndarray, fix: np.ndarray, pix: np.ndarray, cpar: ControlPar):
    # cal = cal_flat.reshape(cal.shape)  # Reshape the flattened calibration parameters
    return cost_function(flat_cal, fix, pix, cpar)

# Flatten the initial calibration parameters
def flat_calibration(cal: Calibration):
    # print(cal.ext_par)
    # print(cal.int_par)
    # print(cal.added_par)
    return np.array([cal.ext_par.x0, cal.ext_par.y0, cal.ext_par.z0, 
            cal.ext_par.omega, cal.ext_par.phi, cal.ext_par.kappa,
            cal.int_par.xh, cal.int_par.yh, cal.int_par.cc, 
            cal.added_par.k1, cal.added_par.k2, cal.added_par.k3, 
            cal.added_par.p1, cal.added_par.p2,
            cal.added_par.scx, cal.added_par.she])
    
initial_cal_flat = flat_calibration(cal)

In [41]:
cost_function(initial_cal_flat, ref_pts, targets, control)

215469108.02916348

In [42]:
# minimize(optimize_function, initial_cal_flat, args=(ref_pts, targets, control), method='Nelder-Mead', tol=1e-3, options={'disp': True})


In [48]:

# Use Scipy's minimize function
result = minimize(optimize_function, initial_cal_flat, args=(ref_pts, targets, control), method='L-BFGS-B',tol=1e-4, options={'disp': True})

# Reshape the result to obtain the optimized calibration parameters
optimized_cal_flat = result.x
# optimized_cal = optimized_cal_flat.reshape(cal.shape)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           16     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.15469D+08    |proj g|=  5.07887D+20


 This problem is unconstrained.



At iterate    1    f=  4.07248D+07    |proj g|=  5.02021D+20



 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



At iterate    2    f=  4.07248D+07    |proj g|=  5.02021D+20

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   16      2     42      2     0     0   5.020D+20   4.072D+07
  F =   40724783.437369630     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


In [44]:
print(cal)

Calibration(ext_par=Exterior: x0=120.2632, y0=87.7458, z0=418.8822
omega=-0.2383291, phi=0.244281, kappa=0.0552577
, int_par=Interior(xh=-2.4742, yh=3.2567, cc=100.0), glass_par=Glass(vec_x=0.0001, vec_y=1e-05, vec_z=150.0), added_par=ap_52(k1=0.0, k2=0.0, k3=0.0, p1=0.0, p2=0.0, scx=1.0, she=0.0), mmlut=mm_lut(origin=array([0., 0., 0.]), nr=0, nz=0, rw=0, data=None))


In [45]:
print(initial_cal_flat - optimized_cal_flat)

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  1.47948675e-19  3.27157385e-16  5.84302988e-11
 -8.72898975e-21 -4.20889534e-21  0.00000000e+00 -2.10925059e-23]


In [46]:
print(unflat_cal(optimized_cal_flat))

Calibration(ext_par=Exterior: x0=120.2632, y0=87.7458, z0=418.8822
omega=-0.2383291, phi=0.244281, kappa=0.0552577
, int_par=Interior(xh=-2.4742, yh=3.2567, cc=100.0), glass_par=Glass(vec_x=0, vec_y=0, vec_z=1), added_par=ap_52(k1=-1.4794867492626326e-19, k2=-3.271573848305543e-16, k3=-5.843029880743216e-11, p1=8.72898974564643e-21, p2=4.208895339140038e-21, scx=1.0, she=2.10925059388362e-23), mmlut=mm_lut(origin=array([0., 0., 0.]), nr=0, nz=0, rw=0, data=None))


In [47]:
cost_function(optimized_cal_flat, ref_pts, targets, control)

40724783.43736963